In [53]:
import numpy as np
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
pd.options.display.max_columns = 100
pd.set_option('display.max_colwidth', -1)

In [8]:
grid_df = pd.read_csv('./output/grid_20230111_200538_withoutInfra.csv')

In [12]:
house_v1 = pd.read_csv('./data/house_v1.csv')
house_v2 = pd.read_csv('./data/house_v2.csv')

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (33,59,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3098: DtypeWarning: Columns (33,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
h1 = house_v1[house_v1['status']=='open']
h2 = house_v2[house_v2['status']=='open']
h = pd.concat([h1,h2])
h = h[~h['item_id'].duplicated()]

In [51]:
tmp = h[['item_id','sales_type','service_type','images','image_thumbnail','매매금액','보증금액','월세금액','전용면적_m2','jibunAddress','local1','local2','title','description','random_location','floor','floor_all','manage_cost','bathroom_count']]

In [55]:
for index, row in tqdm(tmp.iterrows(),total=tmp.shape[0]):
    tmp.loc[index,'위도'] = float(row['random_location'].split(',')[0])
    tmp.loc[index,'경도'] = float(row['random_location'].split(',')[1])

100%|██████████| 12514/12514 [00:14<00:00, 872.51it/s]


In [96]:
for index, row in tqdm(tmp.iterrows(), total=tmp.shape[0]):
    idx = grid_df[( grid_df['min_lat'] < row['위도'] ) & ( row['위도'] < grid_df['max_lat'] )  & ( grid_df['min_long'] < row['경도'] ) & ( row['경도'] < grid_df['max_long'] )].index[0]
    tmp.loc[index,['grid_id']] = grid_df.iloc[idx]['id']

100%|██████████| 12514/12514 [03:02<00:00, 68.45it/s]


In [111]:
tt = tmp[['item_id','경도','위도','grid_id','sales_type','service_type','전용면적_m2','매매금액','보증금액','월세금액','manage_cost','jibunAddress','local1','local2','title','description','image_thumbnail','images','floor','floor_all','bathroom_count']]

In [114]:
tt['register_date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
tt['update_date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
tt['sold_yn'] = 'N'

In [130]:
tt['description'] = tt['description'].apply(lambda x: x.replace("\r\n","<br>"))
tt['description'] = tt['description'].apply(lambda x: x.replace("\n","<br>"))

In [136]:
tt.to_csv('DB_house_info.csv',index=False)